In [1]:
import nltk

In [2]:
import threading

In [3]:
import queue

In [4]:
import feedparser

In [5]:
import uuid

In [6]:
threads = []

In [7]:
queues = [queue.Queue(), queue.Queue()]

In [8]:
def extractWords():
    url = 'https://timesofindia.indiatimes.com/rssfeeds/1081479906.cms'
    feed = feedparser.parse(url)
    for entry in feed['entries'][:5]:
        text = entry['title']
        if 'ex' in text:
            continue
        words = nltk.word_tokenize(text)
        data = {'uuid': uuid.uuid4(), 'input': words}
        queues[0].put(data, True)
        print(">> {} : {}".format(data['uuid'], text))

In [9]:
def extractPOS():
    while True:
        if queues[0].empty():
            break
        else:
            data = queues[0].get()
            words = data['input']
            postags = nltk.pos_tag(words)
            queues[0].task_done()
            queues[1].put({'uuid': data['uuid'], 'input': postags}, True)

In [10]:
def extractNE():
    while True:
        if queues[1].empty():
            break
        else:
            data = queues[1].get()
            postags = data['input']
            queues[1].task_done()
            chunks = nltk.ne_chunk(postags, binary=False)
            print("  << {} : ".format(data['uuid']), end = '')
            for path in chunks:
                try:
                    label = path.label()
                    print(path, end=', ')
                except:
                    pass
            print()

In [11]:
def runProgram():
    e = threading.Thread(target=extractWords())
    e.start()
    threads.append(e)

    p = threading.Thread(target=extractPOS())
    p.start()
    threads.append(p)

    n = threading.Thread(target=extractNE())
    n.start()
    threads.append(n)

    queues[0].join()
    queues[1].join()

    for t in threads:
        t.join()

In [12]:
if __name__ == '__main__':
    runProgram()


>> 45a763e5-5a66-47c6-bc4f-a3999d12a4db : Tanushree accuses Nana Patekar of harassment
>> dfd98bf1-a9d7-4d73-aef1-5e4dac0ed7c1 : Janhvi to star opposite Varun in Rannbhoomi?
>> 29305822-75b6-4175-acaa-ed50ce84e7a5 : Pic: Priyanka Chopra chills like a diva
  << 45a763e5-5a66-47c6-bc4f-a3999d12a4db : (PERSON Nana/NNP Patekar/NNP), 
  << dfd98bf1-a9d7-4d73-aef1-5e4dac0ed7c1 : (GPE Janhvi/NNP), (GPE Rannbhoomi/NNP), 
  << 29305822-75b6-4175-acaa-ed50ce84e7a5 : 
